In [1]:
#wordnet in nltk
from nltk.corpus import wordnet as wn
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score
#import classification report
from sklearn.metrics import classification_report, confusion_matrix
nltk.download('wordnet')
import numpy as np
import gensim
from nltk.data import find

from torch import nn
import torch
from torch.nn import functional as F

#random forest
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package wordnet to /home/dorin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#read imdb/imdb.neg
with open('imdb/imdb.neg', 'r') as f:
    neg = f.readlines()
#read imdb/imdb.pos
with open('imdb/imdb.pos', 'r') as f:
    pos = f.readlines()

#load nltk word2vec_sample
#download nltk model
# nltk.download('word2vec_sample')
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [3]:
# given model and two words, find cosine similarity

def cosine_similarity(model, word1, word2):
    vec1 = model[word1]
    vec2 = model[word2]
    similarity = vec1 @ vec2 / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    #similarity = model.similarity(word1, word2)
    return similarity

In [4]:
# use ngrams to develop a word predictor
# given a word, predict the next word

def ngram_model(corpus, ngram_size):
    # create all the ngrams present in the corpus of size ngram_size
    ngrams = {}
    for line in corpus:
        tokens = line.split()
        for i in range(len(tokens) - ngram_size):
            ngram = tuple(tokens[i:i+ngram_size])
            if ngram not in ngrams:
                ngrams[ngram] = 1
            else:
                ngrams[ngram] += 1
    return ngrams


In [5]:
ngrams = ngram_model(neg+pos, 3)

In [6]:
# # given a word, predict the next word
# def predict_next_word(word, ngrams):
#     # find all ngrams that start with the given word
#     candidates = []
#     for ngram in ngrams:
#         if ngram[0] == word:
#             candidates.append(ngram)
#     # find the most common ngram
#     max_count = 0
#     max_ngram = None
#     for candidate in candidates:
#         if ngrams[candidate] > max_count:
#             max_count = ngrams[candidate]
#             max_ngram = candidate
#     return max_ngram[1]

In [7]:
# def predict_next_word(words, ngrams):
#     #get how many words in ngram
#     ngram_size = len(ngrams.keys()[0])
#     # find all ngrams that start with the given words
#     ngrams_list = list(ngrams.keys())
#     candidates = []
#     for ngram in ngrams_list:
#         if ngram[0:ngram_size] == words:
#             candidates.append(ngram)

#     # find the most common ngram
#     max_count = 0
#     max_ngram = None
#     for candidate in candidates:
#         if ngrams[candidate] > max_count:
#             max_count = ngrams[candidate]
#             max_ngram = candidate
#     return max_ngram[ngram_size]


In [8]:
# def predict_next_n_words(word, ngrams, n):
#     sentence = [word]
#     for i in range(n):
#         word = predict_next_word(sentence, ngrams)
#         sentence.append(word)
#     return " ".join(sentence)

In [9]:
# predict_next_n_words("terrible", ngrams, 5)

In [10]:
# def predict_next_word(vectorizer, word):
#     # get the index of the word in the vocabulary
#     word_idx = vectorizer.vocabulary_.get(word, -1)
#     if word_idx == -1:
#         return None
#     # count the amount of times word is present in the vectorizer as the first element of the ngram
#     word_count = vectorizer.transform([word]).toarray()[0][word_idx]
#     # get the ngrams that start with the word
#     ngrams = vectorizer.get_feature_names()[word_idx:word_idx + 100]
#     # get the counts of the ngrams
#     ngram_counts = vectorizer.transform([word]).toarray()[0][word_idx:word_idx + 100]
#     # get the next word
#     next_word = ngrams[np.argmax(ngram_counts)]
#     return next_word

# predict_next_word(vectorizer, 'forget')

In [11]:
# get the amount of times "good" appears in the vectorizer


In [12]:
cosine_similarity(model, "civilized", "French")

0.08657605

In [64]:
# RNN for sentiment analysis classification
amount_of_data = 15000
test_amount_of_data = 1000
train_data = neg[:amount_of_data] + pos[:amount_of_data]
test_data = neg[amount_of_data:test_amount_of_data+amount_of_data] + pos[amount_of_data:test_amount_of_data+amount_of_data]
train_labels = [0] * amount_of_data + [1] * amount_of_data
test_labels = [0] * test_amount_of_data + [1] * test_amount_of_data

In [65]:
# embed the sentences using word2vec
def embed_sentences(sentences, model):
    embedded_sentences = []
    for sentence in sentences:
        embedded_sentence = []
        for word in sentence.split():
            if word in model:
                embedded_sentence.append(model[word])
        embedded_sentences.append(embedded_sentence)
    return embedded_sentences

embedded_train_data = embed_sentences(train_data, model)
embedded_test_data = embed_sentences(test_data, model)

longest_sentence = max(embedded_train_data, key=len)

# convert sentences to tensor
def pad_and_convert_sentences_to_tensor_padded(embedded_sentences):
    # make hard copy of embedded sentences
    embedded_sentences = [sentence.copy() for sentence in embedded_sentences]
    # pad the sentences
    for sentence in embedded_sentences:
        while len(sentence) < len(longest_sentence):
            sentence.append(np.zeros(300))
    # convert the sentences to tensors
    tensor_sentences = []
    for sentence in embedded_sentences:
        tensor_sentences.append(torch.tensor(sentence))
    tensor_sentences = torch.stack(tensor_sentences).float()
    return tensor_sentences

tensor_train_data = pad_and_convert_sentences_to_tensor_padded(embedded_train_data)
tensor_test_data = pad_and_convert_sentences_to_tensor_padded(embedded_test_data)


In [66]:
#train sentence lengths
train_sentence_lengths = np.array([len(sentence) for sentence in embedded_train_data])
#test sentence lengths
test_sentence_lengths = np.array([len(sentence) for sentence in embedded_test_data])

In [67]:
train_sentence_lengths[20:30]

array([ 1,  1,  4,  4,  6,  5, 10,  3,  4,  0])

In [68]:
temp_data  = torch.Tensor(np.arange(1000).reshape(10,10,10))
indices = torch.Tensor(np.array([1,2,3,4,5,6,7,8,9,4]))
# for all rows, select the dimension=1 of indices and keep the dimension=2
temp_data[np.arange(10), indices.long(), :].shape

torch.Size([10, 10])

In [72]:
# convert labels to float
tensor_train_labels = torch.tensor(train_labels).float()
tensor_test_labels = torch.tensor(test_labels).float()

In [74]:
# create RNN class for the classification task
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        # initialize the layers
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.hidden_size = hidden_size


    def forward(self, x, x_lengths):
        #initialize hidden state
        hidden = torch.zeros(1, x.shape[0], self.hidden_size)
        # run the RNN layer
        #out, _ = self.rnn(x, hidden)
        out, _ = self.rnn(x, (hidden, hidden))
        # get the last hidden state
        #self.hidden = out[:, x_lengths - 1, :]
        hidden = out[np.arange(x.shape[0]), x_lengths-1, :]
        out = self.fc(hidden)
        out = torch.sigmoid(out)
        # drop the extra dimension
        out = out.squeeze(1)
        return out

In [75]:
# create the RNN
rnn = RNN(300, 100, 1)

# define the loss function
loss_function = nn.BCELoss()
# define the optimizer
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001)

# train the RNN
epochs = 10
batch_size = 256

#shuffle tensor_train_data, tensor_train_labels, train_sentence_lengths in the same order
indices = np.arange(len(tensor_train_data))
np.random.shuffle(indices)
tensor_train_data = tensor_train_data[indices]
tensor_train_labels = tensor_train_labels[indices]
train_sentence_lengths = train_sentence_lengths[indices]

for epoch in range(epochs):
    total_loss = 0
    rights = 0
    # shuffle the data
    indices = np.arange(len(tensor_train_data))
    np.random.shuffle(indices)
    tensor_train_data = tensor_train_data[indices]
    tensor_train_labels = tensor_train_labels[indices]   
    train_sentence_lengths = train_sentence_lengths[indices] 
    for i in range(0, len(tensor_train_data), batch_size):
        # get the batch
        batch_data = tensor_train_data[i:i+batch_size]
        #get the original sentence
        batch_lengths = train_sentence_lengths[i:i+batch_size]
        batch_lengths = torch.tensor(batch_lengths).long()
        #get labels
        batch_labels = tensor_train_labels[i:i+batch_size]

        # zero the gradients
        optimizer.zero_grad()
        # run the model
        output = rnn(batch_data, batch_lengths)
        # calculate the loss
        loss = loss_function(output, batch_labels)
        # backpropagate
        loss.backward()
        # update the weights
        optimizer.step()
        total_loss += loss.item()
        rights += torch.sum((output > 0.5) == (batch_labels == 1))



    print("Epoch: {}, Loss: {}".format(epoch, total_loss))
    print("Accuracy: {}".format(rights / len(tensor_train_data)))

Epoch: 0, Loss: 71.41370213031769
Accuracy: 0.684066653251648
Epoch: 1, Loss: 60.285817086696625
Accuracy: 0.7485666871070862
Epoch: 2, Loss: 57.81121703982353
Accuracy: 0.7557333111763
Epoch: 3, Loss: 56.3461237847805
Accuracy: 0.7604666948318481
Epoch: 4, Loss: 55.41433531045914
Accuracy: 0.7629333138465881
Epoch: 5, Loss: 54.517460733652115
Accuracy: 0.7673333287239075
Epoch: 6, Loss: 53.62975910305977
Accuracy: 0.7696333527565002
Epoch: 7, Loss: 52.84335100650787
Accuracy: 0.7719666957855225
Epoch: 8, Loss: 52.43234995007515
Accuracy: 0.7757333517074585
Epoch: 9, Loss: 51.730362981557846
Accuracy: 0.7776333093643188


In [63]:
indices

array([ 7280, 14603, 17482, ..., 10387, 27623, 29573])

In [55]:
batch_lengths

tensor([ 9,  4,  8,  3,  9,  4,  2,  3,  4,  9,  1,  4,  5,  4,  5,  2,  6,  5,
         4,  7,  6,  6,  6,  5,  1,  2, 10,  4,  3,  2,  2,  6,  4,  6,  5, 10,
         3,  2,  0,  1,  6,  3,  4,  3,  7,  5,  5,  3,  3,  3,  7,  0,  1,  1,
         2,  2,  4, 11,  2,  7,  5, 10,  6,  5,  2,  6,  3, 10,  0,  3,  7,  3,
         1,  5,  2,  2,  6,  2,  1,  7,  5,  4,  5,  7,  4, 10,  9,  6, 10,  4,
         5,  6,  3,  7,  5,  1,  8,  7, 10,  9,  7,  4,  4,  2,  0,  1,  3,  8,
         6,  0,  3,  2,  2,  4,  8,  4,  1,  1,  6,  6,  6,  1,  1,  4,  2,  8,
         6,  0,  2,  8, 10, 14,  7,  4,  1,  2,  4,  8,  9,  6,  3,  2,  4,  7,
         6,  3,  9,  3,  9,  3,  8,  2,  2,  7,  3,  7,  8,  9,  0,  2,  1,  0,
         3,  4,  9,  6,  1,  4,  3,  0,  9,  3,  3,  7,  6,  1,  6,  0,  7,  4,
         5,  1,  2,  7,  5,  0,  3,  6,  4,  6,  4,  2,  6,  3,  3,  3,  4,  5,
         3,  6, 13,  4,  8,  5,  2,  7,  2,  2,  0,  2,  2,  0,  8,  2,  2,  6,
         0,  3,  0,  4,  3,  2,  4,  4, 

In [27]:
batch_labels

tensor([1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1.])

In [30]:
# test the RNN
predictions = []
for i in range(len(tensor_test_data)):
    # get the sentence
    sentence = tensor_test_data[i]
    # get the sentence length
    sentence_length = test_sentence_lengths[i]
    # get the label
    label = tensor_test_labels[i]
    # run the model
    output = rnn(sentence.unsqueeze(0), torch.tensor([sentence_length]).long())
    # get the prediction
    prediction = round(output.item())
    predictions.append(prediction)

# calculate the accuracy
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy: {}".format(accuracy))



Accuracy: 0.7389


In [137]:
tensor_test_labels

tensor([0., 0., 0.,  ..., 1., 1., 1.])